# Customer Single View

In [ ]:
%pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

###Import raw data

In [ ]:
path = 'https://github.com/AsmaMora/MADT8101/raw/main/1.Customer_single_view&Basic_customer_anlytics/supermarket.parquet'

In [ ]:
df_ = pd.read_parquet(path)

In [ ]:
raw = df_

###Sanity check and clean raw data

In [ ]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578082 entries, 0 to 578081
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SHOP_WEEK                 578082 non-null  int64  
 1   SHOP_DATE                 578082 non-null  int64  
 2   SHOP_WEEKDAY              578082 non-null  int64  
 3   SHOP_HOUR                 578082 non-null  int64  
 4   QUANTITY                  578082 non-null  int64  
 5   SPEND                     578082 non-null  float64
 6   PROD_CODE                 578082 non-null  object 
 7   PROD_CODE_10              578082 non-null  object 
 8   PROD_CODE_20              578082 non-null  object 
 9   PROD_CODE_30              578082 non-null  object 
 10  PROD_CODE_40              578082 non-null  object 
 11  CUST_CODE                 492494 non-null  object 
 12  CUST_PRICE_SENSITIVITY    492494 non-null  object 
 13  CUST_LIFESTAGE            433142 non-null  o

In [ ]:
# Find member and non member info
non_member_trans = raw[raw['CUST_CODE'].isnull()]['SHOP_DATE'].count()
member_trans = raw[raw['CUST_CODE'].notnull()]['SHOP_DATE'].count()
total_trans = non_member_trans + member_trans
non_member_spend = raw[raw['CUST_CODE'].isnull()]['SPEND'].sum()
member_spend = raw[raw['CUST_CODE'].notnull()]['SPEND'].sum()
member = raw[raw['CUST_CODE'].notnull()]['CUST_CODE'].nunique()

# Print the results
print("total_trans:", total_trans)
print("member_trans:", member_trans)
print("non_member_trans:", non_member_trans)
print("member_spend:", member_spend)
print("non_member_spend:", non_member_spend)
print("member:", member)

total_trans: 578082
member_trans: 492494
non_member_trans: 85588
member_spend: 949467.93
non_member_spend: 168460.39
member: 3439


In [ ]:
raw.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200734,20071019,6,8,1,9.50,PRD0901335,CL00058,DEP00016,G00006,...,XX,OT,994108000679877,M,MM,Small Shop,Fresh,STORE00001,LS,E02
1,200805,20080328,6,21,1,1.49,PRD0901120,CL00162,DEP00055,G00016,...,XX,None,994110300629787,S,MM,Small Shop,Fresh,STORE00001,LS,E02
2,200716,20070612,3,9,1,0.86,PRD0900963,CL00248,DEP00089,G00030,...,None,None,994106200026009,S,MM,Small Shop,Grocery,STORE00001,LS,E02
3,200815,20080604,4,10,3,0.90,PRD0901055,CL00248,DEP00089,G00030,...,None,None,994111300076346,L,MM,Top Up,Grocery,STORE00001,LS,E02
4,200626,20060824,5,16,1,1.77,PRD0901877,CL00169,DEP00056,G00017,...,None,None,994102000065854,L,LA,Top Up,Fresh,STORE00001,LS,E02


Check missing value

In [ ]:
#check missing value
raw.isnull().sum()

SHOP_WEEK                        0
SHOP_DATE                        0
SHOP_WEEKDAY                     0
SHOP_HOUR                        0
QUANTITY                         0
SPEND                            0
PROD_CODE                        0
PROD_CODE_10                     0
PROD_CODE_20                     0
PROD_CODE_30                     0
PROD_CODE_40                     0
CUST_CODE                    85588
CUST_PRICE_SENSITIVITY       85588
CUST_LIFESTAGE              144940
BASKET_ID                        0
BASKET_SIZE                      0
BASKET_PRICE_SENSITIVITY         0
BASKET_TYPE                      0
BASKET_DOMINANT_MISSION          0
STORE_CODE                       0
STORE_FORMAT                     0
STORE_REGION                     0
dtype: int64

Remove missing value

In [ ]:
# remove rows where CUST_ID is 0
raw = raw.dropna(subset=['CUST_CODE'], how='any', axis=0)

Change data type

In [ ]:
#change data type
raw['SHOP_DATE'] = pd.to_datetime(raw['SHOP_DATE'], format='%Y%m%d')

In [ ]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492494 entries, 0 to 578081
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SHOP_WEEK                 492494 non-null  int64         
 1   SHOP_DATE                 492494 non-null  datetime64[ns]
 2   SHOP_WEEKDAY              492494 non-null  int64         
 3   SHOP_HOUR                 492494 non-null  int64         
 4   QUANTITY                  492494 non-null  int64         
 5   SPEND                     492494 non-null  float64       
 6   PROD_CODE                 492494 non-null  object        
 7   PROD_CODE_10              492494 non-null  object        
 8   PROD_CODE_20              492494 non-null  object        
 9   PROD_CODE_30              492494 non-null  object        
 10  PROD_CODE_40              492494 non-null  object        
 11  CUST_CODE                 492494 non-null  object        
 12  CU

###Prepare label column
create label for 3 columns


*   CUST_PRICE_SENSITIVITY
*   CUST_LIFESTAGE
*   BASKET_SIZE



In [ ]:
#create label columns
def map_price_sensitivity(sensitivity):
    if sensitivity == "LA": return "Less Affluent"
    elif sensitivity == "MM": return "Mid Market"
    elif sensitivity == "UM": return "UP Market"
    else: return "Unclas"


raw['CUST_PRICE_SENSITIVITY_LABEL'] = raw['CUST_PRICE_SENSITIVITY'].apply(map_price_sensitivity)


def map_cust_lifestage(lifestage):
    if lifestage == "YA": return "Young Adults"
    elif lifestage == "OA": return "OA"
    elif lifestage == "YF": return "Young Families"
    elif lifestage == "OF": return "Older Adults"
    elif lifestage == "PE": return "Pensioners"
    else: return "Other"


raw['CUST_LIFESTAGE_LABEL'] = raw['CUST_LIFESTAGE'].apply(map_cust_lifestage)


def map_basket_size(size):
    if size == "L": return "Large"
    elif size == "M": return "Medium"
    else: return "Small"


raw['BASKET_SIZE_LABEL'] = raw['BASKET_SIZE'].apply(map_basket_size)


#create code columns
def map_price_sensitivity_cd(sensitivity_cd):
    if sensitivity_cd == "LA": return 0
    elif sensitivity_cd == "MM": return 1
    elif sensitivity_cd == "UM": return 2
    else: return 3


raw['CUST_PRICE_SENSITIVITY_CD'] = raw['CUST_PRICE_SENSITIVITY'].apply(map_price_sensitivity_cd)


def map_cust_lifestage_cd(lifestage_cd):
    if lifestage_cd == "YA": return 0
    elif lifestage_cd == "OA": return 1
    elif lifestage_cd == "YF": return 2
    elif lifestage_cd == "OF": return 3
    elif lifestage_cd == "PE": return 4
    else: return 5


raw['CUST_LIFESTAGE_CD'] = raw['CUST_LIFESTAGE'].apply(map_cust_lifestage_cd)


def map_basket_size_cd(size_cd):
    if size_cd == "L": return 0
    elif size_cd == "M": return 1
    else: return 2


raw['BASKET_SIZE_CD'] = raw['BASKET_SIZE'].apply(map_basket_size_cd)

In [ ]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492494 entries, 0 to 578081
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   SHOP_WEEK                     492494 non-null  int64         
 1   SHOP_DATE                     492494 non-null  datetime64[ns]
 2   SHOP_WEEKDAY                  492494 non-null  int64         
 3   SHOP_HOUR                     492494 non-null  int64         
 4   QUANTITY                      492494 non-null  int64         
 5   SPEND                         492494 non-null  float64       
 6   PROD_CODE                     492494 non-null  object        
 7   PROD_CODE_10                  492494 non-null  object        
 8   PROD_CODE_20                  492494 non-null  object        
 9   PROD_CODE_30                  492494 non-null  object        
 10  PROD_CODE_40                  492494 non-null  object        
 11  CUST_CODE    

In [ ]:
raw.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION,CUST_PRICE_SENSITIVITY_LABEL,CUST_LIFESTAGE_LABEL,BASKET_SIZE_LABEL,CUST_PRICE_SENSITIVITY_CD,CUST_LIFESTAGE_CD,BASKET_SIZE_CD
0,200734,2007-10-19,6,8,1,9.50,PRD0901335,CL00058,DEP00016,G00006,...,Fresh,STORE00001,LS,E02,Unclas,Other,Medium,3,5,1
1,200805,2008-03-28,6,21,1,1.49,PRD0901120,CL00162,DEP00055,G00016,...,Fresh,STORE00001,LS,E02,Unclas,Other,Small,3,5,2
1115,200815,2008-06-05,5,15,1,0.70,PRD0904509,CL00093,DEP00028,G00008,...,Grocery,STORE00001,LS,E02,Less Affluent,Older Adults,Large,0,3,0
1116,200642,2006-12-13,4,15,1,2.04,PRD0904730,CL00214,DEP00071,G00022,...,Fresh,STORE00001,LS,E02,Less Affluent,Older Adults,Large,0,3,0
1117,200720,2007-07-13,6,17,1,2.19,PRD0900777,CL00098,DEP00031,G00008,...,Mixed,STORE00001,LS,E02,Less Affluent,OA,Large,0,1,0


### Create customer single view

In [ ]:
# Define aggregation functions
aggregations = {
    'SHOP_DATE': ['min', 'max'],
    'BASKET_ID': 'nunique',
    'SPEND': 'sum'
}

# Group by specified columns and apply aggregations
summary = raw.groupby(['CUST_CODE','CUST_LIFESTAGE_LABEL', 'CUST_LIFESTAGE_CD','CUST_PRICE_SENSITIVITY_LABEL', 'CUST_PRICE_SENSITIVITY_CD']).agg(aggregations).reset_index()

# Rename columns for clarity
summary.columns = ['CUST_CODE','CUST_LIFESTAGE_LABEL', 'CUST_LIFESTAGE_CD','CUST_PRICE_SENSITIVITY_LABEL', 'CUST_PRICE_SENSITIVITY_CD',
                   'Start_Date', 'Last_Date', 'FQ', 'Total_Spend']

# Calculate additional columns
reference_date = pd.to_datetime('2008-07-15')
summary['MTBP'] = (summary['Last_Date'] - summary['Start_Date']).dt.days / summary['FQ']
summary['Life_Time'] = (reference_date - summary['Start_Date']).dt.days
summary['ARPU'] = summary['Total_Spend'] / summary['FQ']
summary['CLTV'] = (reference_date - summary['Start_Date']).dt.days * (summary['Total_Spend'] / summary['FQ'])

# Calculate counts for specific columns and divide by 'FQ'
columns_to_count = ['BASKET_SIZE', 'BASKET_PRICE_SENSITIVITY', 'BASKET_TYPE', 'BASKET_DOMINANT_MISSION']
for column in columns_to_count:
    pivot_df = raw.pivot_table(index='CUST_CODE', columns=column, values='BASKET_ID', aggfunc='nunique', fill_value=0).reset_index()
    pivot_columns = [f'{column}_{value}' for value in pivot_df.columns[1:]]
    pivot_df.columns = ['CUST_CODE'] + pivot_columns
    summary = summary.merge(pivot_df, on='CUST_CODE', how='left')
    # Divide by 'FQ'
    summary[pivot_columns] = summary[pivot_columns].div(summary['FQ'], axis=0)




In [ ]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3439 entries, 0 to 3438
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   CUST_CODE                        3439 non-null   object        
 1   CUST_LIFESTAGE_LABEL             3439 non-null   object        
 2   CUST_LIFESTAGE_CD                3439 non-null   int64         
 3   CUST_PRICE_SENSITIVITY_LABEL     3439 non-null   object        
 4   CUST_PRICE_SENSITIVITY_CD        3439 non-null   int64         
 5   Start_Date                       3439 non-null   datetime64[ns]
 6   Last_Date                        3439 non-null   datetime64[ns]
 7   FQ                               3439 non-null   int64         
 8   Total_Spend                      3439 non-null   float64       
 9   MTBP                             3439 non-null   float64       
 10  Life_Time                        3439 non-null   int64      

In [ ]:
summary.head()

,CUST_CODE,CUST_LIFESTAGE_LABEL,CUST_LIFESTAGE_CD,CUST_PRICE_SENSITIVITY_LABEL,CUST_PRICE_SENSITIVITY_CD,Start_Date,Last_Date,FQ,Total_Spend,MTBP,...,BASKET_PRICE_SENSITIVITY_XX,BASKET_TYPE_Full Shop,BASKET_TYPE_Small Shop,BASKET_TYPE_Top Up,BASKET_TYPE_XX,BASKET_DOMINANT_MISSION_Fresh,BASKET_DOMINANT_MISSION_Grocery,BASKET_DOMINANT_MISSION_Mixed,BASKET_DOMINANT_MISSION_Nonfood,BASKET_DOMINANT_MISSION_XX
0,CUST0000000181,Other,5,Mid Market,1,2007-01-06,2007-01-06,1,2.44,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0
1,CUST0000000689,Other,5,Less Affluent,0,2007-07-22,2008-06-23,16,328.57,21.062500,...,0.000000,0.437500,0.312500,0.250000,0.0,0.312500,0.187500,0.437500,0.062500,0.0
2,CUST0000000998,Other,5,Unclas,3,2006-05-04,2006-07-07,3,5.95,21.333333,...,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.666667,0.000000,0.333333,0.0
3,CUST0000001163,Other,5,Mid Market,1,2006-10-22,2008-06-22,4,39.74,152.250000,...,0.000000,0.000000,0.500000,0.500000,0.0,0.750000,0.000000,0.250000,0.000000,0.0
4,CUST0000001194,Other,5,Mid Market,1,2006-04-22,2008-06-20,103,2493.83,7.669903,...,0.009709,0.330097,0.184466,0.485437,0.0,0.533981,0.097087,0.349515,0.019417,0.0
